In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

x_tensile_normal_train = np.load('D:/wangshuo/paper4/tensile_train_normal_0dB.npy') 

x_shear_normal_train = np.load('D:/wangshuo/paper4/shear_train_normal_0dB.npy')  

x_tensile_truck_train = np.load('D:/wangshuo/paper4/tensile_train_truck_0dB.npy')

x_shear_truck_train = np.load('D:/wangshuo/paper4/shear_train_truck_0dB.npy')

X_train = np.vstack([
    x_tensile_normal_train,
    x_shear_normal_train,
    x_tensile_truck_train,
    x_shear_truck_train
])

y_tensile_normal_train = np.load('D:/wangshuo/paper4/la_all.npy') 
y_shear_normal_train = np.load('D:/wangshuo/paper4/jian_all.npy') 
y_tensile_truck_train = np.load('D:/wangshuo/paper4/la_all.npy') 
y_shear_truck_train = np.load('D:/wangshuo/paper4/jian_all.npy') 

Y_train = np.vstack([
    y_tensile_normal_train,
    y_shear_normal_train,
    y_tensile_truck_train,
    y_shear_truck_train
])

x_train, x_val, y_train, y_val = train_test_split(
    X_train, Y_train,
    test_size=0.1,    
    random_state=42,   
    shuffle=True     
)



x_train = x_train[..., np.newaxis]  # (N, 1024, 1)
y_train = y_train[..., np.newaxis]

x_val = x_val[..., np.newaxis]  # (N, 1024, 1)
y_val = y_val[..., np.newaxis]


from tensorflow.keras import layers, models


def conv_block(x, filters, kernel_size=3, activation='relu'):
    x = layers.Conv1D(filters, kernel_size, padding='same')(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv1D(filters, kernel_size, padding='same')(x)
    x = layers.Activation(activation)(x)
    return x

def encoder_block(x, filters):
    x = conv_block(x, filters)
    p = layers.MaxPooling1D(pool_size=2)(x)
    return x, p

def decoder_block(x, skip, filters):
    x = layers.UpSampling1D(size=2)(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x



def build_unet_1d(input_length=1024):
    inputs = layers.Input(shape=(input_length, 1))
    e1, p1 = encoder_block(inputs, 32)
    e2, p2 = encoder_block(p1, 64)
    e3, p3 = encoder_block(p2, 128)
    e4, p4 = encoder_block(p3, 256)
    e5, p5 = encoder_block(p4, 512)
    e6, p6 = encoder_block(p5, 1024)
    e7, p7 = encoder_block(p6, 1024)
    
    b = conv_block(p7, 1024)
    

    d7 = decoder_block(b, e7, 1024)
    d6 = decoder_block(d7, e6, 1024)
    d5 = decoder_block(d6, e5, 512)
    d4 = decoder_block(d5, e4, 256)
    d3 = decoder_block(d4, e3, 128)
    d2 = decoder_block(d3, e2, 64)
    d1 = decoder_block(d2, e1, 32)
    outputs = layers.Conv1D(1, kernel_size=1, activation='linear')(d1)
    return models.Model(inputs=inputs, outputs=outputs)


model = build_unet_1d(input_length=1024)
model.summary()

from tensorflow.keras.callbacks import EarlyStopping


model.compile(optimizer='adam', loss='mse', metrics=['mae'])

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = 'D:/wangshuo/paper4/denoise_7block_0dB.h5'


checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False, 
    verbose=1
)


history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=100,
    batch_size=2000,
    callbacks=[checkpoint], 
    verbose=1
)


history_path = 'D:/wangshuo/paper4/training_curve_7block_0dB.txt'
with open(history_path, 'w') as f:
    f.write("Epoch\tLoss\tVal_Loss\tMAE\tVal_MAE\n")
    for i in range(len(history.history['loss'])):
        f.write(f"{i+1}\t{history.history['loss'][i]:.6f}\t"
                f"{history.history['val_loss'][i]:.6f}\t"
                f"{history.history['mae'][i]:.6f}\t"
                f"{history.history['val_mae'][i]:.6f}\n")

print(f" {history_path}")

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


base_path = r"D:\wangshuo\paper4"
x0_path = os.path.join(base_path, "jian_all.npy") 
x1_path = os.path.join(base_path, "la_all.npy")   


tf.random.set_seed(42)
np.random.seed(42)


x0 = np.load(x0_path)  # class 0
x1 = np.load(x1_path)  # class 1

# 若是一维，扩成二维 (N, 1)
if x0.ndim == 1: x0 = x0[:, None]
if x1.ndim == 1: x1 = x1[:, None]
assert x0.shape[1] == x1.shape[1],  {x0.shape[1]} vs {x1.shape[1]}"


X = np.vstack([x0, x1]).astype(np.float32)
y = np.concatenate([np.zeros(len(x0), dtype=np.int32),
                    np.ones(len(x1),  dtype=np.int32)])


X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.10,
    random_state=42,
    shuffle=True,
    stratify=y
)

def build_simple_mlp(input_dim: int) -> tf.keras.Model:
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(1024, activation='relu')(inputs)
    
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(1024, activation='relu')(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)  # 二分类
    return models.Model(inputs, outputs)

model = build_simple_mlp(X_train.shape[1])
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)
model.summary()

best_model_path = os.path.join(base_path, "clf_3layer.h5")
ckpt = ModelCheckpoint(
    filepath=best_model_path,
    monitor='val_auc', mode='max',
    save_best_only=True, verbose=1
)
es = EarlyStopping(
    monitor='val_auc', mode='max',
    patience=100, restore_best_weights=True, verbose=1
)


history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=2000,
    callbacks=[ckpt, es],
    verbose=1
)

eval_res = model.evaluate(X_val, y_val, verbose=0)
print("\nValidation Metrics:")
for name, val in zip(model.metrics_names, eval_res):
    print(f"{name}: {val:.4f}")


curve_path = os.path.join(base_path, "clf_3layer_training_curve.txt")
with open(curve_path, "w") as f:
    keys = list(history.history.keys())
    f.write("epoch\t" + "\t".join(keys) + "\n")
    for i in range(len(history.history[keys[0]])):
        row = [str(i+1)] + [f"{history.history[k][i]:.6f}" for k in keys]
        f.write("\t".join(row) + "\n")

